In [1]:
import os
import tensorflow as tf
from PIL import Image

# create binary data func

In [2]:
def create_record():
    cwd = os.getcwd()
    classes = {'train','test'}
    writer = tf.python_io.TFRecordWriter("train.tfrecords")
    for index, name in enumerate(classes):
        class_path = cwd + "/" + name +"/"
        for img_name in os.listdir(class_path):
            img_path = class_path + img_name
            img = Image.open(img_path)
            img = img.resize((64, 64))
            # transform img to bytes
            img_raw = img.tobytes() 
            print index,img_raw
            example = tf.train.Example(
               features=tf.train.Features(feature={
                    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
               }))
            writer.write(example.SerializeToString())
    writer.close()

# read binary data func

In [3]:
def read_and_decode(filename):
    # build file queue
    filename_queue = tf.train.string_input_producer([filename])
    # create a reader from file queue
    reader = tf.TFRecordReader()
    # reade serialized sample
    _, serialized_example = reader.read(filename_queue)
    # get feature from serialized example
    features = tf.parse_single_example(serialized_example,
        features={'label': tf.FixedLenFeature([], tf.int64),
            'img_raw': tf.FixedLenFeature([], tf.string)})
    label = features['label']
    img = features['img_raw']
    img = tf.decode_raw(img, tf.uint8)
    img = tf.reshape(img, [64, 64, 3])
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5
    label = tf.cast(label, tf.int32)
    return img, label

# create and read binary data

In [4]:
if __name__ == '__main__':
    
    data = create_record()
    
    if "train.tfrecords":
        img, label = read_and_decode("train.tfrecords")
        print "image matrix: ",img,label
        # shuffle_batch randomly input
        # shuffle_batch synchronize [img,label]
        # shuffle_batch returns RandomShuffleQueue.dequeue_many()
        # Shuffle_batch builds a RandomShuffleQueue， and put [img,label] into the queue
        img_batch, label_batch = tf.train.shuffle_batch([img, label],
            batch_size=4, capacity=2000, min_after_dequeue=1000)

        # init
        init = tf.global_variables_initializer()
        
        with tf.Session() as sess:
            sess.run(init)
            # start queue
            threads = tf.train.start_queue_runners(sess=sess)
            for i in range(5):
                print img_batch.shape,label_batch
                val, l = sess.run([img_batch, label_batch])
                # l = to_categorical(l, 12)
                print(val.shape, l)

��.��2��@��?��F��:��M��R��3��;��E��J��=��F��?��K��I��?��3��1��9��2y�-v�.t�,q�)lzWiwTguTbpO`nM_mJaoLepNgrPlwUq|Zx�`�d��j��r��x��������������������ý�����±�ĳ�Ƕ�˺�ͼ�����������������W��(��$��?��W��;���S��P��;��I��3��-��B��C��B��E��7��E��D��8��0��.��8��3}�1x�0v�.q�)ivXfsUboS^kO_lP^lI_mJcnLepNkvTp{Yv�^�d��i��p��v��������������������ý�����±�ĳ�ȷ�˺�ͼ�����������˽����)����4��K��?��@��M��J��M��N��G��5��B��B��C��H��>��*��6��>��6��&��*��2��4�3{�3w�/q�)duUarR`mO^kM^kM^kM]jL`mOboQitVny[r�]{�c��j��q��v���������������������Ľ�ƿ��Į�Ķ�ƶ����������������ָ��Q������@��O��+��8��5��M��J��U��7��;��C��C��G��D��:��3��2��0��1��'��&��+��0��2|�/x�.r�*arR^oO^kM\iK\iK^kM]jL_lNanPgrTmxZp~[z�b��h��o��v���������������������Ľ�ƿ��ð�Ǵ�ȵ�̹�Ϲ��������������,��	��'��<��[��2��'��1��?��=��[��2��7��P��<��<��9��A��=��7��-��1��,��*��-��/��1��4}�1u�-_pP\mM\iK\iK\iK[hJ]jL_lNanPepRlwYo}Zy�a��g��n��u���������������������ü�����ó�Ƴ�ȼ���༢ܸ��ȭ�ӹ��Q������)��6��L��6��8��9��>��.��O��E��0��0��9��0��2��K��@��:��1��1��7��1��2��-��1��2}�.

(4, 64, 64, 3) Tensor("shuffle_batch:1", shape=(4,), dtype=int32)
((4, 64, 64, 3), array([1, 1, 0, 0], dtype=int32))
(4, 64, 64, 3) Tensor("shuffle_batch:1", shape=(4,), dtype=int32)
((4, 64, 64, 3), array([1, 1, 1, 0], dtype=int32))
(4, 64, 64, 3) Tensor("shuffle_batch:1", shape=(4,), dtype=int32)
((4, 64, 64, 3), array([1, 1, 1, 1], dtype=int32))
(4, 64, 64, 3) Tensor("shuffle_batch:1", shape=(4,), dtype=int32)
((4, 64, 64, 3), array([0, 1, 0, 1], dtype=int32))
(4, 64, 64, 3) Tensor("shuffle_batch:1", shape=(4,), dtype=int32)
((4, 64, 64, 3), array([1, 0, 0, 0], dtype=int32))
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: shuffle_batch/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch/random_shuffle_queue, sub/_7, Cast_1/_9)]]

Caused by op u'shuffle_batch/random_shuffle_queue_enqueue', defined at:
  File "/usr/lib/python2.7/runpy.py", line

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/training/queue_runner_impl.py", line 234, in _run
    sess.run(enqueue_op)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 778, in run
    run_metadata_ptr)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 982, in _run
    feed_dict_string, options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1032, in _do_run
    target_list, options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1052, in _do_call
    raise type(e)(node_def, op, message)
CancelledError: Enqueue operati